- Langchain
- Pipeline
- Agents
- Chains 
- Face

# Models Pipeline 

In [1]:
import transformers
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification # for medical ner classification
from transformers import AutoModelForMaskedLM 

2024-04-16 14:28:43.159029: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Medical NER

In [7]:
mer_pipe = pipeline(task="token-classification",
                    model="Clinical-AI-Apollo/Medical-NER",
                    aggregation_strategy = "simple"
            )

mer_tokenizer = AutoTokenizer.from_pretrained("Clinical-AI-Apollo/Medical-NER")

mer_model = AutoModelForTokenClassification.from_pretrained("Clinical-AI-Apollo/Medical-NER")

In [17]:
mer_pipe = pipeline(task="token-classification", 
                    model=mer_model, 
                    tokenizer=mer_tokenizer
            )

mer_text = """

show me the report where patient has hypertensiona and lisinopril is prescribed, he has pain but no vomiting

"""

mer_text = mer_text.lower()

print(mer_text)

tokenized_text_mer = mer_tokenizer(text=mer_text, return_tensors="pt") # pytorch format 
masx_length_mer = 512

print(tokenized_text_mer)

# the pipeline has tokenizer and the llm
mer_result = mer_pipe(mer_text)

print(mer_result)



show me the report where patient has hypertensiona and lisinopril is prescribed, he has pain but no vomiting


{'input_ids': tensor([[    1,   553,   351,   262,   815,   399,  1799,   303, 17088,   452,
           263, 68314,   269,  8048,   261,   313,   303,  1427,   304,   363,
         17730,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
[{'entity': 'B-DISEASE_DISORDER', 'score': 0.50351614, 'index': 8, 'word': '▁hypertension', 'start': 38, 'end': 51}, {'entity': 'I-DISEASE_DISORDER', 'score': 0.21287218, 'index': 9, 'word': 'a', 'start': 51, 'end': 52}, {'entity': 'B-MEDICATION', 'score': 0.7583509, 'index': 11, 'word': '▁lisinopril', 'start': 56, 'end': 67}, {'entity': 'B-SIGN_SYMPTOM', 'score': 0.88620543, 'index': 17, 'word': '▁pain', 'start': 89, 'end': 94}, {'entity': 'B-SIGN_SYMPTOM', 'score': 0.68255216, 'index': 20, 

In [4]:
def format_ner_output(result, text):
    entity_dict = {}
    current_type = None  
    current_start = None  
    flag = 0

    for entity in result:
        entity_type = entity["entity"][2:]
        start = entity["start"]
        end = entity["end"]
        word = text[start:end]

        if current_type == entity_type and current_start is not None:
            flag += 1
            entity_dict[text[current_start:end]] = entity_type  
        else:
            print(flag) if flag != 0 else None
            flag = 0
            entity_dict[word] = entity_type  
            current_type = entity_type
            current_start = start

    return entity_dict

formatted_output_dict = format_ner_output(mer_result, mer_text)

for key, value in formatted_output_dict.items():
    print(f'{key}   -   {value}\n')


4
2
2
2
1
3
4
1
1
1
1
1
2
2
1
1
7
3
9
3
 44   -   AGE

 44-   -   AGE

 44-year   -   AGE

 44-year-   -   AGE

 44-year-old   -   AGE

 male   -   SEX

 hypertension   -   DISEASE_DISORDER

Li   -   MEDICATION

Lisino   -   MEDICATION

Lisinopril   -   MEDICATION

 20   -   DOSAGE

 20mg   -   DOSAGE

 20mg daily   -   DOSAGE

 presents   -   CLINICAL_EVENT

 2   -   DURATION

 2-   -   DURATION

 2-day   -   DURATION

 sharp   -   DETAILED_DESCRIPTION

 sharp, 
cramping   -   DETAILED_DESCRIPTION

 right   -   BIOLOGICAL_STRUCTURE

 right lower   -   BIOLOGICAL_STRUCTURE

 right lower quadrant   -   BIOLOGICAL_STRUCTURE

 right lower quadrant abdominal   -   BIOLOGICAL_STRUCTURE

 pain   -   SIGN_SYMPTOM

 pain. He denies fever   -   SIGN_SYMPTOM

 pain. He denies fever, nausea   -   SIGN_SYMPTOM

 pain. He denies fever, nausea, vomiting   -   SIGN_SYMPTOM

 pain. He denies fever, nausea, vomiting, diarrhea   -   SIGN_SYMPTOM

 bowel   -   HISTORY

 habit   -   SIGN_SYMPTOM

 habit c

In [7]:
import pickle

with open('output_dict.pkl', 'wb') as f:
  pickle.dump(formatted_output_dict, f)

In [8]:

with open('output_dict.pkl', 'rb') as f:
  loaded_dict = pickle.load(f)


loaded_dict

{' 44': 'AGE',
 ' 44-': 'AGE',
 ' 44-year': 'AGE',
 ' 44-year-': 'AGE',
 ' 44-year-old': 'AGE',
 ' male': 'SEX',
 ' hypertension': 'DISEASE_DISORDER',
 'Li': 'MEDICATION',
 'Lisino': 'MEDICATION',
 'Lisinopril': 'MEDICATION',
 ' 20': 'DOSAGE',
 ' 20mg': 'DOSAGE',
 ' 20mg daily': 'DOSAGE',
 ' presents': 'CLINICAL_EVENT',
 ' 2': 'DURATION',
 ' 2-': 'DURATION',
 ' 2-day': 'DURATION',
 ' sharp': 'DETAILED_DESCRIPTION',
 ' sharp, \ncramping': 'DETAILED_DESCRIPTION',
 ' right': 'BIOLOGICAL_STRUCTURE',
 ' right lower': 'BIOLOGICAL_STRUCTURE',
 ' right lower quadrant': 'BIOLOGICAL_STRUCTURE',
 ' right lower quadrant abdominal': 'BIOLOGICAL_STRUCTURE',
 ' pain': 'SIGN_SYMPTOM',
 ' pain. He denies fever': 'SIGN_SYMPTOM',
 ' pain. He denies fever, nausea': 'SIGN_SYMPTOM',
 ' pain. He denies fever, nausea, vomiting': 'SIGN_SYMPTOM',
 ' pain. He denies fever, nausea, vomiting, diarrhea': 'SIGN_SYMPTOM',
 ' bowel': 'HISTORY',
 ' habit': 'SIGN_SYMPTOM',
 ' habit changes': 'SIGN_SYMPTOM',
 ' Examinati

### Conversation

In [ ]:
conv_tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")

conv_model = AutoModelForMaskedLM.from_pretrained("medicalai/ClinicalBERT")


For conversation, 

- Rule based and AI-powered chatbot 
- 
## Dialogue Management System


### RAG docs